# NLP for Fake News detection
Applying Neural Language Processing to try and identify fake news from 'true' news. 

Due to tokenization process, its indicate to replicate this code on a Google Colab environment.

Dataset availabke at: https://www.kaggle.com/clmentbisaillon/fake-and-real-news-dataset 

### Data exploration

In [1]:
# importing necessary libs
import pandas as pd
import numpy as np

In [2]:
# reading csv into a dataframe
fake_news = pd.read_csv('Fake.csv')
true_news = pd.read_csv('True.csv')

In [3]:
true_news.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [4]:
fake_news.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [5]:
# number of columns and rows of each dataframe
fake_news.shape, true_news.shape

((23481, 4), (21417, 4))

In [6]:
fake_news.describe().T

,count,unique,top,freq
title,23481,17903,MEDIA IGNORES Time That Bill Clinton FIRED His...,6
text,23481,17455,,626
subject,23481,6,News,9050
date,23481,1681,"May 10, 2017",46


In [7]:
true_news.describe().T

,count,unique,top,freq
title,21417,20826,Factbox: Trump fills top jobs for his administ...,14
text,21417,21192,(Reuters) - Highlights for U.S. President Dona...,8
subject,21417,2,politicsNews,11272
date,21417,716,"December 20, 2017",182


In [8]:
# adding feature that will be target variable True/Fake
fake_news['True/Fake'] = 'True'
true_news['True/Fake'] = 'Fake'

In [9]:
# combining the dataframes into a single one using concat method
total_news = pd.concat([true_news, fake_news])
total_news['Article'] = total_news['title'] + total_news['text'] # merging columns title and text

In [10]:
# shuffling dataset 100% in order to mix True and Fake
total_news.sample(frac=1)

,title,text,subject,date,True/Fake,Article
4020,Congress readies votes Friday on bill to avert...,WASHINGTON (Reuters) - The U.S. Congress is se...,politicsNews,"April 27, 2017",Fake,Congress readies votes Friday on bill to avert...
4844,Highlights: The Trump presidency on March 19 a...,(Reuters) - Highlights of the day for U.S. Pre...,politicsNews,"March 19, 2017",Fake,Highlights: The Trump presidency on March 19 a...
16457,JUDGE JEANINE Tells Us What The Smoking Gun Is...,Judge Jeanine gives us facts on the relationsh...,Government News,"Aug 12, 2016",True,JUDGE JEANINE Tells Us What The Smoking Gun Is...
11138,YIKES! JUST WHEN Everyone Thought Things Could...,"It s been a bumpy PR ride for United Airlines,...",politics,"Apr 14, 2017",True,YIKES! JUST WHEN Everyone Thought Things Could...
4087,Factbox: U.S. Congress investigates links betw...,(Reuters) - Republican leaders in the U.S. Con...,politicsNews,"April 25, 2017",Fake,Factbox: U.S. Congress investigates links betw...
...,...,...,...,...,...,...
20243,BREAKING: HILLARY MAKES DIVERSE PICK FOR VP….W...,So much for that whole diversity thing I'm thr...,left-news,"Jul 22, 2016",True,BREAKING: HILLARY MAKES DIVERSE PICK FOR VP….W...
10948,BREAKING REPORT: ANTIFA THUGS PLACE BOUNTY On ...,The next time you hear someone talk about how ...,politics,"May 7, 2017",True,BREAKING REPORT: ANTIFA THUGS PLACE BOUNTY On ...
23109,SUNDAY SCREENING: 24 Hours After Hiroshima (2010),21st Century Wire says This week s documentary...,Middle-east,"December 11, 2016",True,SUNDAY SCREENING: 24 Hours After Hiroshima (20...
5563,Trump's Supreme Court pick dispirited by presi...,WASHINGTON (Reuters) - Donald Trump’s Supreme ...,politicsNews,"February 6, 2017",Fake,Trump's Supreme Court pick dispirited by presi...


#### Basic Data Cleaning

In [14]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\leticiag\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [15]:
# importing libs
from nltk.corpus import stopwords
import string

In [1]:
# build function to process text
def process_text(s):

    # Check each string for punctuation
    no_punc = [char for char in s if char not in string.punctuation]

    # Join the characters again 
    no_punc = ''.join(no_punc)
    
    # Convert string to lowercase and remove stopwords
    clean_string = [word for word in no_punc.split() if word.lower() not in stopwords.words('english')]
    return clean_string

In [17]:
# Tokenize article so words can be represented by numbers
# Takes 3h avg on jupyter notebook, 30 minutes on Google Colab
total_news['Clean Text'] = total_news['Article'].apply(process_text)

In [18]:
total_news.sample(5)

,title,text,subject,date,True/Fake,Article,Clean Text
14695,"China, Philippines agree to avoid force in Sou...",BEIJING/MANILA (Reuters) - China and the Phili...,worldnews,"November 16, 2017",Fake,"China, Philippines agree to avoid force in Sou...","[China, Philippines, agree, avoid, force, Sout..."
15982,BURN! SEBASTIAN GORKA HUMILIATES MSNBC On Use ...,It s probably not the wisest decision to argue...,Government News,"Aug 8, 2017",True,BURN! SEBASTIAN GORKA HUMILIATES MSNBC On Use ...,"[BURN, SEBASTIAN, GORKA, HUMILIATES, MSNBC, Us..."
18673,Syrian army and allies secure road after IS at...,BEIRUT (Reuters) - The Syrian army and its all...,worldnews,"September 29, 2017",Fake,Syrian army and allies secure road after IS at...,"[Syrian, army, allies, secure, road, attack, D..."
3151,Trump likely to nominate former Senate aide Pe...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"June 16, 2017",Fake,Trump likely to nominate former Senate aide Pe...,"[Trump, likely, nominate, former, Senate, aide..."
1958,Trump Official Accidentally Leaks Plan To Ste...,Mexican President Enrique Pe a Nieto may not b...,News,"March 29, 2017",True,Trump Official Accidentally Leaks Plan To Ste...,"[Trump, Official, Accidentally, Leaks, Plan, S..."


#### Vectorization
Convert the collection of text documents to a matrix of token counts. 

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

In [20]:
# Bag-of-Words (bow) transform the entire DataFrame of text
# transforming tokens into useful features
bow_transformer = CountVectorizer(analyzer=process_text).fit(total_news['Clean Text'])

In [22]:
print(f'Total vocabulary words: {len(bow_transformer.vocabulary_)}') 

Total vocabulary words: 39099


In [24]:
news_bow = bow_transformer.transform(total_news['Clean Text'])

In [25]:
# printing out the shape of our Matrix
print(f'Shape of Sparse Matrix: {news_bow.shape}')
print(f'Amount of Non-Zero occurences: {news_bow.nnz}')

Shape of Sparse Matrix: (44898, 39099)
Amount of Non-Zero occurences: 44898


In [27]:
# checking out sparsity
sparsity = (100.0 * news_bow.nnz / (news_bow.shape[0] * news_bow.shape[1]))
print(f'sparsity: {round(sparsity)}')

sparsity: 0


In [28]:
# extracting text with Transformer
from sklearn.feature_extraction.text import TfidfTransformer

In [29]:
# checking new shape
tfidf_transformer = TfidfTransformer().fit(news_bow)
news_tfidf = tfidf_transformer.transform(news_bow)
print(news_tfidf.shape)

(44898, 39099)


#### Build and train the model

In [30]:
# importing algorithm
from sklearn.naive_bayes import MultinomialNB

In [31]:
fakenews_detect_model = MultinomialNB().fit(news_tfidf, total_news['True/Fake'])

In [32]:
# Model Evaluation
predictions = fakenews_detect_model.predict(news_tfidf)
print(predictions)

['Fake' 'Fake' 'Fake' ... 'True' 'True' 'True']


In [33]:
# importing metrics to check preditions
from sklearn.metrics import classification_report
print (classification_report(total_news['True/Fake'], predictions))

              precision    recall  f1-score   support

        Fake       1.00      1.00      1.00     21417
        True       1.00      1.00      1.00     23481

    accuracy                           1.00     44898
   macro avg       1.00      1.00      1.00     44898
weighted avg       1.00      1.00      1.00     44898



#### Building pipeline

In [35]:
# importing train_test_split to split the data and train the model
from sklearn.model_selection import train_test_split

news_train, news_test, text_train, text_test = train_test_split(total_news['Article'], total_news['True/Fake'], test_size=0.3)

In [36]:
# building pipeline
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=process_text)),  
    ('tfidf', TfidfTransformer()),  
    ('classifier', MultinomialNB()),  
])
pipeline.fit(news_train,text_train)

Pipeline(memory=None,
         steps=[('bow',
                 CountVectorizer(analyzer=<function process_text at 0x000000E6D4B5C8B0>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('classifier',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

### Evaluating the model

In [37]:
# predicting news on test dataset
prediction = pipeline.predict(news_test)

In [38]:
# printing out classification report
print(classification_report(prediction,text_test))

              precision    recall  f1-score   support

        Fake       0.97      0.96      0.96      6621
        True       0.96      0.98      0.97      6849

    accuracy                           0.97     13470
   macro avg       0.97      0.97      0.97     13470
weighted avg       0.97      0.97      0.97     13470



The mode has a high precision detecting both fake and true news. We have high precision, high recall and high f1-score. 